In [3]:
# imports
from config import dictionaryloc
from config import turntextloc
from config import wheeltextloc
from config import maxrounds
from config import vowelcost
from config import roundstatusloc
from config import finalprize
from config import finalRoundTextLoc

import random

# initalize variables
players={0:{"roundtotal":0,"gametotal":0,"name":""},
         1:{"roundtotal":0,"gametotal":0,"name":""},
         2:{"roundtotal":0,"gametotal":0,"name":""},
        }

roundNum = 0
dictionary = []
turntext = ""
wheellist = []
roundWord = ""
blankWord = []
vowels = {"a", "e", "i", "o", "u"}
roundstatus = ""
finalroundtext = ""
letter_track = []   # tracks letters guessed 
                    # list instead of set, to index for grammar -- see tracker()

# functions
def readDictionaryFile():
    global dictionary

    # Read dictionary file in from dictionary file location
    temp_read = open(dictionaryloc)
    dictionary.append(temp_read.read().splitlines())
    temp_read.close()

    # Store each word in a list.
    # a list of list is created by splitlines(), so eliminate a layer
    dictionary = dictionary[0]


def readTurnTxtFile():
    global turntext

    #read in turn intial turn status "message" from file
    temp_read = open(turntextloc)
    turntext = temp_read.read()
    temp_read.close()

        
def readFinalRoundTxtFile():
    global finalroundtext   

    #read in turn intial turn status "message" from file
    temp_read = open(finalRoundTextLoc)
    finalroundtext = temp_read.read()
    temp_read.close()


def readRoundStatusTxtFile():
    global roundstatus

    # read the round status using the Config roundstatusloc file location
    temp_read = open(roundstatusloc)
    roundstatus = temp_read.read()
    temp_read.close()


def readWheelTxtFile():
    global wheellist

    # read the Wheel name from input using the Config wheelloc file location
    temp_read = open(wheeltextloc)
    wheellist.append(temp_read.read().splitlines())
    temp_read.close()

    # store each outcome in a list
    # splitlines() creates a list of list, so eliminate a layer
    wheellist = wheellist[0]


def getPlayerInfo():
    global players

    # read in player names from command prompt input
    player_number = 0
    while player_number < 3: # loops through all 3 players
        player_name = str(input('Enter the name of player number {0}'.format(player_number + 1)))
        players[player_number]["name"] = player_name.strip()
        player_number += 1


def divider(): # divider for readability
    print('===============================================')


def reassembleguess(): # returns what the board looks like with guessed letters and _s
    global blankWord

    reass_guess = ''
    for x in blankWord:
        reass_guess += x

    return reass_guess


def tracker(): # prints what letters have been guessed so far from 'letter_track'
    global letter_track
    track_string = ''

    for x in letter_track:
        if len(letter_track) == 1:
            track_string = x.upper()
        # uses indices to find last entry and appends an 'and'
        elif letter_track.index(x) != len(letter_track) - 1: 
            track_string += x.upper() + ", "
        else:
            track_string += "and " + x.upper()

    print('So far, the guessed letters are: {0}'.format(track_string))


def gameSetup():
    # Read in File dictionary
    # Read in Turn Text Files
    global turntext
    global dictionary
    
    # all txt files are read and stored in global strings
    readDictionaryFile()
    readTurnTxtFile()
    readWheelTxtFile()
    getPlayerInfo()
    readRoundStatusTxtFile()
    readFinalRoundTxtFile()


def getWord():
    global dictionary

    #choose random word from dictionary list
    roundWord = random.choice(dictionary)

    #make a list of the word with underscores instead of letters.
    roundUnderscoreWord = []
    answer_length = len(roundWord)

    while answer_length != 0:
        roundUnderscoreWord.append('_')
        answer_length -= 1

    return roundWord,roundUnderscoreWord # local scope, will need assignment


def wofRoundSetup():
    global players
    global roundWord
    global blankWord
    global letter_track

    # actions for new round

    # clear tracked letters
    letter_track = []

    # adding 'roundtotal' to 'gametotal'
    for player_number in players:
        players[player_number]['gametotal'] += players[player_number]['roundtotal']

    # Set round total for each player = 0
    for player_number in players:
        players[player_number]['roundtotal'] = 0

    # Return the starting player number (random)
    player_list = [0, 1, 2]
    initPlayer = random.choice(player_list)

    # Use getWord function to retrieve the word and the underscore word (blankWord)
    roundWord, blankWord = getWord()

    return initPlayer


def spinWheel(playerNum):
    global wheellist
    global players
    global vowels

    # Get random value for wheellist
    wheel_outcome = random.choice(wheellist)
    print('The wheel lands on {0}!'.format(wheel_outcome))

    # Check for bankrupcy, and take action.
    if wheel_outcome == 'BANKRUPT':
        players[playerNum]['roundtotal'] = 0
        stillinTurn = False

    # Check for loose turn
    elif wheel_outcome == 'LOSE A TURN':
        stillinTurn = False

    # Get amount from wheel if not loose turn or bankruptcy
    else:
        dollar_value = wheel_outcome.split('$')[1]
    
        # Ask user for letter guess
        guess_length = 0
        while guess_length != 1 or letter_guess in vowels:
            letter_guess = str(input('Enter a single letter to guess: ')).strip()
            guess_length = len(letter_guess)

            if guess_length != 1:
                print("That isn't a single letter!")

            # ensure letter is a consonate.
            if letter_guess in vowels:
                print('The letter must be a consonant!')

            # require only alphabet characters - stretch goal

        # Use guessletter function to see if guess is in word, and return count
        stillinTurn, count = guessletter(letter_guess, playerNum)
        
        # Change player round total if they guess right.
        cash = int(dollar_value) * int(count)
        players[playerNum]['roundtotal'] += cash

    return stillinTurn


def guessletter(letter, playerNum): 
    global players
    global blankWord
    global roundWord
    global vowels
    global letter_track

    # parameters:  take in a letter guess and player number
    
    multi_index = 0
    search_start = 0
    
    # return goodGuess= true if it was a correct guess
    # keep track of letters guessed

    new_letter = False
    
    while not new_letter: 
    # loops until player guesses a new letter
    # so players can't exit a wheel outcome by entering an old letter
        letter = letter.lower()

        if letter in letter_track or letter == '' or len(letter) != 1:  
            if letter in letter_track: # letter already guessed
                print("That letter's already been guessed!")

            else: # blank letter / not a letter
                print("Invalid input.")

            tracker() # reminds player what's been guessed already
            letter = str(input('Enter a single letter to guess: ')).strip()

            new_letter = False

        elif letter not in letter_track: # new letter
            letter_track.append(letter)
            new_letter = True
            indices = []

            # loops through and finds all occurrences of letter
            while multi_index != -1: 
                multi_index = roundWord.find(letter, search_start)
                if multi_index != -1:
                    search_start = multi_index + 1
                    indices.append(multi_index) # stores occurrences' indices into list

            if not indices: # if letter is not found, then 'indices' is empty
                print('Letter "{0}" not found.'.format(letter.upper()))
                goodGuess = False

            else: # letter found
                goodGuess = True
                print('Letter "{0}" found!'.format(letter.upper()))
                # Change position of found letter in blankWord to the letter instead of underscore 
                for x in indices:
                    blankWord[x] = letter # replaces underscore(s) with letter(s)
                
                reass_guess = reassembleguess()
                print("The Board: {0}\n".format(reass_guess))

            count = len(indices)

    # return count of letters in word. 

    return goodGuess, count


def buyVowel(playerNum):
    global players
    global vowels
    global letter_track
    guess_length = 0
    vowel_guess = ''
    # Take in a player number
    
    # checks if vowels have been exhausted yet
    if all(x in letter_track for x in vowels):
        print('All vowels have been exhausted!')
        goodGuess = True # player chooses another option

    # Ensure player has 250 for buying a vowelcost
    elif players[playerNum]['roundtotal'] >= vowelcost:
        players[playerNum]['roundtotal'] -= vowelcost

        # Ensure valid vowel input
        while guess_length != 1 or vowel_guess not in vowels or vowel_guess in letter_track:
            print('What vowel would you like to buy?')
            vowel_guess = str(input('Enter a vowel to guess: ')).strip().lower()
            guess_length = len(vowel_guess)

            if vowel_guess in letter_track: # vowel already guessed
                print('That vowel has already been guessed!')
            if vowel_guess not in vowels: # guess is not a vowel
                print('The letter must be a vowel!')
            if guess_length != 1: # guess is not a single letter
                print("That isn't a single letter!")
            
        # Use guessLetter function to see if the letter is in the file
        # If letter is in the file let goodGuess = True
        goodGuess, count = guessletter(vowel_guess, playerNum)
    
    else: # player can't afford vowel
        print("You don't have enough money to buy a vowel!\n")
        goodGuess = True # player is still in their turn
                         # must choose another option -- see wofTurn()
    
    return goodGuess


def guessWord(playerNum):
    global players
    global blankWord
    global roundWord
    
    # Take in player number
    # Ask for input of the word and check if it is the same as roundword
    word_guess = str(input('What word would you like to guess?: ')).strip().lower()
    if word_guess == roundWord:
        print('Congratulations! The word was "{0}"!'.format(roundWord))
        # Fill in blankWord with all letters, instead of underscores if correct
        blankWord = list(roundWord)

    else:
        print("Sorry, '{0}' isn't the word. :(".format(word_guess))

    # return False ( to indicate the turn will finish)  
    return False


def wofTurn(playerNum):  
    global roundWord
    global blankWord
    global turntext
    global players
    global roundstatus
    # take in a player number. 

    # use the string.format method to output your status for the round
    roundstart = roundstatus.splitlines()[0].format(roundNum + 1)
    print(roundstart)
    
    # and Ask to (s)pin the wheel, (b)uy vowel, or G(uess) the word using
    print('What would you like to do?\n(S)pin the wheel\n(B)uy a vowel\n(G)uess the word\n(L)eaderboard\n(T)rack the letters guessed')
    divider()

    # Keep doing all turn activity for a player until they guess wrong
    # Do all turn related activity including update roundtotal
    stillinTurn = True
    while stillinTurn:
        
        # use the string.format method to output your status for the round
        # Get user input S for spin, B for buy a vowel, G for guess the word
        choice = str(input('Please input your choice: '))
                
        if(choice.strip().upper() == "S"):
            stillinTurn = spinWheel(playerNum)

        elif(choice.strip().upper() == "B"):
            stillinTurn = buyVowel(playerNum)

        elif(choice.strip().upper() == "G"):
            stillinTurn = guessWord(playerNum)

        elif(choice.strip().upper() == "L"):
            print('Leaderboards:')
            for x in players:
                pnum = players[x]
                print('{0}:\n\tRound Bank: {1}\n\tTotal Bank: {2}\n'
                    .format(pnum['name'],pnum['roundtotal'],pnum['gametotal']))

        elif(choice.strip().upper() == "T"):
            tracker()

        else:
            print("Not a correct option.\n")        
    
    # Check to see if the word is solved, and return false if it is,
    # Or otherwise break the while loop of the turn.


def wofRound():
    global players
    global roundWord
    global blankWord
    global roundstatus
    global letter_track

    playerNum = wofRoundSetup()
    
    # Keep doing things in a round until the round is done ( word is solved)
        # While still in the round keep rotating through players
        # Use the wofTurn fuction to dive into each players turn until their turn is done.
    while '_' in blankWord:
        divider()
        print(turntext.format(players[playerNum]["name"], players[playerNum]["roundtotal"]))

        print("\nThe Board: " + reassembleguess() + "\n")

        wofTurn(playerNum)
        playerNum += 1
        if playerNum == 3:
            playerNum = 0 # loops around
    # Print roundstatus with string.format, tell people the state of the round as you are leaving a round.
    roundend = roundstatus.splitlines()[1].format(roundNum + 1)
    print(roundend)


def wofFinalRound():
    global roundWord
    global blankWord
    global finalroundtext
    global letter_track
    winplayer = 0

    # clears tracked letters + moves 'roundtotal' to 'gametotal'
    wofRoundSetup()

    # Find highest gametotal player.  They are playing.
    player_rankings = sorted(players.items(), key = lambda x: x[1]['gametotal'], reverse = True)
        # sorts players from highest  to lowest 'gametotal' score
    winplayer = player_rankings[0][0]
    winner = player_rankings[0][1]['name'] # Name/number of player with highest gametotalscore

    # Print out instructions for that player and who the player is.
    divider()
    readFinalRoundTxtFile()
    print(finalroundtext.format(winner))

    # Use the getWord function to reset the roundWord and the blankWord ( word with the underscores)
    divider()
    roundWord, blankWord = getWord()

    # Use the guessletter function to check for {'R','S','T','L','N','E'}
    final_round_letters = {'R','S','T','L','N','E'}
    for x in final_round_letters:
        guessletter(x, winplayer)

    # Print out the current blankWord with whats in it after applying {'R','S','T','L','N','E'}
        # done in guessletter()
    
    # Gather 3 consonats and 1 vowel and use the guessletter function to see if they are in the word
    consonant_count = 3
    vowel_count = 1

    while consonant_count != 0 or vowel_count != 0:
        # grammar
        if consonant_count == 1: 
            consonant_string = 'consonant'
        else:
            consonant_string = 'consonants'

        if vowel_count == 1:
            vowel_string = 'vowel'
        else:
            vowel_string = 'vowels'

        # input guess
        letter_guess = str(input("You have {0} {cons_str} and {1} {v_str} left. Guess a letter, or type 'guess' to guess the word: "
            .format(consonant_count, vowel_count, cons_str = consonant_string, v_str = vowel_string)))

        if letter_guess == 'guess': # player wants to guess
            break

        elif len(letter_guess) == 1 or letter_guess != '':  # guess letter

            if letter_guess in letter_track: # old guess
                print("That letter's already been guessed!")
                tracker()

            elif letter_guess in vowels: # vowel chosen
                if vowel_count == 0:
                    print('You have no more vowels left!')
                else:
                    vowel_count -= 1
                    guessletter(letter_guess, winplayer)
            elif letter_guess not in vowels: # consonant chosen
                if consonant_count == 0:
                    print('You have no more consonants left!')
                else:
                    consonant_count -= 1
                    guessletter(letter_guess, winplayer)
        else:
            print('That is not a valid option!')

    # Print out the current blankWord again
    # Remember guessletter should fill in the letters with the positions in blankWord
    reass_guess = reassembleguess()
    print('Final Round Board: {0}\n'.format(reass_guess))

    # Get user to guess word
    guessWord(winplayer)

    # If they do, add finalprize and gametotal and print out that the player won
    if blankWord == list(roundWord):
        print("You've won a cash prize of ${0}!\n".format(finalprize))
        players[winplayer]['gametotal'] += finalprize
    else:
        print("The word was '{0}'.\n".format(roundWord))

    print("Your final game bank is ${0}!".format(players[winplayer]['gametotal']))


def main():
    global roundNum
    gameSetup()    

    for roundNum in range(0,maxrounds):
        if roundNum in [0,1]:
            wofRound()
        else:
            wofFinalRound()
        roundNum += 1


if __name__ == "__main__":
    main()


It is Twoson's turn!
You have $0 in your round bank.

The Board: ______

This is round number 1!
What would you like to do?
(S)pin the wheel
(B)uy a vowel
(G)uess the word
(L)eaderboard
(T)rack the letters guessed
The wheel lands on $650!
Letter "T" found!
The Board: ___t__

What vowel would you like to buy?
Letter "I" not found.
It is Threed's turn!
You have $0 in your round bank.

The Board: ___t__

This is round number 1!
What would you like to do?
(S)pin the wheel
(B)uy a vowel
(G)uess the word
(L)eaderboard
(T)rack the letters guessed
Leaderboards:
Onett:
	Round Bank: 0
	Total Bank: 0

Twoson:
	Round Bank: 400
	Total Bank: 0

Threed:
	Round Bank: 0
	Total Bank: 0

The wheel lands on $550!
Letter "L" found!
The Board: ___tl_

Leaderboards:
Onett:
	Round Bank: 0
	Total Bank: 0

Twoson:
	Round Bank: 400
	Total Bank: 0

Threed:
	Round Bank: 550
	Total Bank: 0

Sorry, 'beetle' isn't the word. :(
It is Onett's turn!
You have $0 in your round bank.

The Board: ___tl_

This is round numbe

IndexError: list assignment index out of range